<a href="https://colab.research.google.com/github/fbelinchon/HuggingFace_ES/blob/main/Analisis_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 325 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 44.3 MB/s 
     |████████████████████████████████| 212 kB 43.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.4 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 134 kB 53.2 MB/s 
     |████████████████████████████████| 127 kB 41.9 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 271 kB 70.6 MB/s 
     |████████████████████████████████| 144 kB 42.6 MB/s 
     |████████████████████████████████| 6.5 MB 45.1 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |████████████████████████████████| 596 kB 37.5 MB/s 
     |████████████████████████████████| 1.2 MB 39.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

# Método simple

In [2]:
from transformers import pipeline

In [ ]:
checkpoint = 'pysentimiento/robertuito-sentiment-analysis'

sentimiento = pipeline('sentiment-analysis',model=checkpoint)

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
texto=['Todo esto es perfecto','De lo malo lo peor','la silla es blanca']

print(sentimiento(texto))

[{'label': 'POS', 'score': 0.9886201620101929}, {'label': 'NEG', 'score': 0.9688207507133484}, {'label': 'NEU', 'score': 0.9894474744796753}]


# Método elaborado

In [3]:
from transformers import  AutoTokenizer,AutoModelForSequenceClassification

In [6]:
checkpoint = 'pysentimiento/robertuito-sentiment-analysis'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [8]:
texto=['Todo esto es perfecto','De lo malo lo peor','la silla es blanca']
batch = tokenizer(texto,padding=True, truncation=True, return_tensors="pt")

print(batch)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[   0,  658,  669,  442, 4976,    2,    1],
        [   0,  413,  496, 3302,  496, 1599,    2],
        [   0,  446, 8925,  442, 3367,    2,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [10]:
sentimiento = model(**batch)

print(sentimiento)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.8312, -0.5093,  4.0487],
        [ 2.9229, -0.6040, -2.9596],
        [-1.2066,  3.6102, -2.3526]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [14]:
from torch.nn.functional import softmax

prediccion = softmax(sentimiento.logits,dim=-1)
print(prediccion)

tensor([[0.0010, 0.0104, 0.9886],
        [0.9688, 0.0285, 0.0027],
        [0.0080, 0.9894, 0.0025]], grad_fn=<SoftmaxBackward0>)


In [17]:
model.config.id2label

{0: 'Negativo', 1: 'Neutro', 2: 'Positivo'}

In [16]:
model.config.id2label = {0: 'Negativo', 1: 'Neutro', 2: 'Positivo'}

In [39]:
import numpy as np

a =np.array(np.argmax(prediccion.detach(),axis=1))

In [69]:
for ind,valores in enumerate(zip(texto,a)):
    print('frase: "{}" con sentimiento {} {:.2f}%'.format(valores[0],model.config.id2label[valores[1]],prediccion.detach()[ind,valores[1]]))

frase: "Todo esto es perfecto" con sentimiento Positivo 0.99%
frase: "De lo malo lo peor" con sentimiento Negativo 0.97%
frase: "la silla es blanca" con sentimiento Neutro 0.99%
